This kernel shows an `ensemble` method in this competition.  

I take the code from two kernels and ensemble their results 

- https://www.kaggle.com/adityaork/decision-tree-smart-data-augmentation
- https://www.kaggle.com/zaharch/arc-c-approach

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from pathlib import Path
from collections import defaultdict
from itertools import product
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations,permutations
from sklearn.tree import *
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
import random
from math import floor

In [ ]:
data_path = Path("/kaggle/input/abstraction-and-reasoning-challenge")
train_path = data_path/'training'
test_path = data_path/'test'

def plot_result(inp,eoup,oup):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, 3, figsize=(15,15))
    
    axs[0].imshow(inp, cmap=cmap, norm=norm)
    axs[0].axis('off')
    axs[0].set_title('Input')

    axs[1].imshow(eoup, cmap=cmap, norm=norm)
    axs[1].axis('off')
    axs[1].set_title('Output')
    
    axs[2].imshow(oup, cmap=cmap, norm=norm)
    axs[2].axis('off')
    axs[2].set_title('Model prediction')
    
    plt.grid()
    plt.tight_layout()
    plt.show()

def plot_mats(mats):
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
    norm = colors.Normalize(vmin=0, vmax=9)
    fig, axs = plt.subplots(1, len(mats), figsize=(15,15))
    
    for i in range(len(mats)):
        axs[i].imshow(mats[i], cmap=cmap, norm=norm)
        axs[i].axis('off')
        axs[i].set_title('Fig: '+str(i))
    
    plt.rc('grid', linestyle="-", color='white')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
def getiorc(pair):
    inp = pair["input"]
    return pair["input"],pair["output"],len(inp),len(inp[0])
    
def getAround(i,j,inp,size=1):
    #v = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    r,c = len(inp),len(inp[0])
    v = []
    sc = [0]
    for q in range(size):
        sc.append(q+1)
        sc.append(-(q+1))
    for idx,(x,y) in enumerate(product(sc,sc)):
        ii = (i+x)
        jj = (j+y)
        v.append(-1)
        if((0<= ii < r) and (0<= jj < c)):
            v[idx] = (inp[ii][jj])
    return v

def getDiagonal(i,j,r,c):
    return
        
    
def getX(inp,i,j,size):
    z = []
    n_inp = np.array(inp)
    z.append(i)
    z.append(j)
    r,c = len(inp),len(inp[0])
    for m in range(5):
        z.append(i%(m+1))
        z.append(j%(m+1))
    z.append(i+j)
    z.append(i*j)
#     z.append(i%j)
#     z.append(j%i)
    z.append((i+1)/(j+1))
    z.append((j+1)/(i+1))
    z.append(r)
    z.append(c)
    z.append(len(np.unique(n_inp[i,:])))
    z.append(len(np.unique(n_inp[:,j])))
    arnd = getAround(i,j,inp,size)
    z.append(len(np.unique(arnd)))
    z.extend(arnd)
    return z

def getXy(inp,oup,size):
    x = []
    y = []
    r,c = len(inp),len(inp[0])
    for i in range(r):
        for j in range(c):
            x.append(getX(inp,i,j,size))
            y.append(oup[i][j])
    return x,y
    
def getBkgColor(task_json):
    color_dict = defaultdict(int)
    
    for pair in task_json['train']:
        inp,oup,r,c = getiorc(pair)
        for i in range(r):
            for j in range(c):
                color_dict[inp[i][j]]+=1
    color = -1
    max_count = 0
    for col,cnt in color_dict.items():
        if(cnt > max_count):
            color = col
            max_count = cnt
    return color

def get_num_colors(inp,oup,bl_cols):
    r,c = len(inp),len(inp[0])
    return 

def replace(inp,uni,perm):
    # uni = '234' perm = ['5','7','9']
    #print(uni,perm)
    r_map = { int(c):int(s) for c,s in zip(uni,perm)}
    r,c = len(inp),len(inp[0])
    rp = np.array(inp).tolist()
    #print(rp)
    for i in range(r):
        for j in range(c):
            if(rp[i][j] in r_map):
                rp[i][j] = r_map[rp[i][j]]
    return rp
            
    
def augment(inp,oup,bl_cols):
    cols = "0123456789"
    npr_map = [1,9,72,3024,15120,60480,181440,362880,362880]
    uni = "".join([str(x) for x in np.unique(inp).tolist()])
    for c in bl_cols:
        cols=cols.replace(str(c),"")
        uni=uni.replace(str(c),"")

    exp_size = len(inp)*len(inp[0])*npr_map[len(uni)]
    
    mod = floor(exp_size/120000)
    mod = 1 if mod==0 else mod
    
    #print(exp_size,mod,len(uni))
    result = []
    count = 0
    for comb in combinations(cols,len(uni)):
        for perm in permutations(comb):
            count+=1
            if(count % mod == 0):
                result.append((replace(inp,uni,perm),replace(oup,uni,perm)))
    return result
            
def get_flips(inp,oup):
    result = []
    n_inp = np.array(inp)
    n_oup = np.array(oup)
    result.append((np.fliplr(inp).tolist(),np.fliplr(oup).tolist()))
    result.append((np.rot90(np.fliplr(inp),1).tolist(),np.rot90(np.fliplr(oup),1).tolist()))
    result.append((np.rot90(np.fliplr(inp),2).tolist(),np.rot90(np.fliplr(oup),2).tolist()))
    result.append((np.rot90(np.fliplr(inp),3).tolist(),np.rot90(np.fliplr(oup),3).tolist()))
    result.append((np.flipud(inp).tolist(),np.flipud(oup).tolist()))
    result.append((np.rot90(np.flipud(inp),1).tolist(),np.rot90(np.flipud(oup),1).tolist()))
    result.append((np.rot90(np.flipud(inp),2).tolist(),np.rot90(np.flipud(oup),2).tolist()))
    result.append((np.rot90(np.flipud(inp),3).tolist(),np.rot90(np.flipud(oup),3).tolist()))
    result.append((np.fliplr(np.flipud(inp)).tolist(),np.fliplr(np.flipud(oup)).tolist()))
    result.append((np.flipud(np.fliplr(inp)).tolist(),np.flipud(np.fliplr(oup)).tolist()))
    return result
    
def gettaskxy(task_json,aug,around_size,bl_cols,flip=True):    
    X = []
    Y = []
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        tx,ty = getXy(inp,oup,around_size)
        X.extend(tx)
        Y.extend(ty)
        if(flip):
            for ainp,aoup in get_flips(inp,oup):
                tx,ty = getXy(ainp,aoup,around_size)
                X.extend(tx)
                Y.extend(ty)
                if(aug):
                    augs = augment(ainp,aoup,bl_cols)
                    for ainp,aoup in augs:
                        tx,ty = getXy(ainp,aoup,around_size)
                        X.extend(tx)
                        Y.extend(ty)
        if(aug):
            augs = augment(inp,oup,bl_cols)
            for ainp,aoup in augs:
                tx,ty = getXy(ainp,aoup,around_size)
                X.extend(tx)
                Y.extend(ty)
    return X,Y

def test_predict(task_json,model,size):
    inp = task_json['test'][0]['input']
    eoup = task_json['test'][0]['output']
    r,c = len(inp),len(inp[0])
    oup = predict(inp,model,size)
    return inp,eoup,oup

def predict(inp,model,size):
    r,c = len(inp),len(inp[0])
    oup = np.zeros([r,c],dtype=int)
    for i in range(r):
        for j in range(c):
            x = getX(inp,i,j,size)
            o = int(model.predict([x]))
            o = 0 if o<0 else o
            oup[i][j]=o
    return oup

def submit_predict(task_json,model,size):
    pred_map = {}
    idx=0
    for pair in task_json['test']:
        inp = pair["input"]
        oup = predict(inp,model,size)
        pred_map[idx] = oup.tolist()
        idx+=1
        plot_result(inp,oup,oup)
    return pred_map

def dumb_predict(task_json):
    pred_map = {}
    idx=0
    for pair in task_json['test']:
        inp = pair["input"]
        pred_map[idx] = [[0,0],[0,0]]
        idx+=1
    return pred_map

def get_loss(model,task_json,size):
    total = 0
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        eoup = predict(inp,model,size)
        total+= np.sum((np.array(oup) != np.array(eoup)))
    return total

def get_test_loss(model,task_json,size):
    total = 0
    for pair in task_json['test']:
        inp,oup=pair["input"],pair["output"]
        eoup = predict(inp,model,size)
        total+= np.sum((np.array(oup) != np.array(eoup)))
    return total

def get_a_size(task_json):
    return 4;

def get_bl_cols(task_json):
    result = []
    bkg_col = getBkgColor(task_json);
    result.append(bkg_col)
    # num_input,input_cnt,num_output,output_cnt
    met_map = {}
    for i in range(10):
        met_map[i] = [0,0,0,0]
        
    total_ex = 0
    for pair in task_json['train']:
        inp,oup=pair["input"],pair["output"]
        u,uc = np.unique(inp, return_counts=True)
        inp_cnt_map = dict(zip(u,uc))
        u,uc = np.unique(oup, return_counts=True)
        oup_cnt_map = dict(zip(u,uc))
        
        for col,cnt in inp_cnt_map.items():
            met_map[col][0] = met_map[col][0] + 1
            met_map[col][1] = met_map[col][1] + cnt
        for col,cnt in oup_cnt_map.items():
            met_map[col][2] = met_map[col][2] + 1
            met_map[col][3] = met_map[col][3] + cnt
        total_ex+=1
    
    for col,met in met_map.items():
        num_input,input_cnt,num_output,output_cnt = met
        if(num_input == total_ex or num_output == total_ex):
            result.append(col)
        elif(num_input == 0 and num_output > 0):
            result.append(col)
    
    result = np.unique(result).tolist()
    if(len(result) == 10):
        result.append(bkg_col)
    return np.unique(result).tolist()

def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

def combine_preds(tid,pm1,pm3,pm5):
    result = []
    for i in range(len(pm1)):
        tk_s = tid+"_"+str(i)
        str_pred = flattener(pm1[i])+" "+flattener(pm3[i])+" "+flattener(pm5[i])
        #print(tk_s,str_pred)
        result.append([tk_s,str_pred])
    return result

def inp_oup_dim_same(task_json):
    return all([ len(pair["input"]) == len(pair["output"]) and len(pair["input"][0]) == len(pair["output"][0])
                for pair in task_json['train']])
    

solved_task = 0
total_task = 0
task_ids = []
task_preds = []
for task_path in test_path.glob("*.json"):
    task_json = json.load(open(task_path))
    tk_id = str(task_path).split("/")[-1].split(".")[0]
    print(tk_id)
    if(inp_oup_dim_same(task_json)):
        a_size = get_a_size(task_json)
        bl_cols = get_bl_cols(task_json)
        
        isflip = False
        X1,Y1 = gettaskxy(task_json,True,1,bl_cols,isflip)
        X3,Y3 = gettaskxy(task_json,True,3,bl_cols,isflip)
        X5,Y5 = gettaskxy(task_json,True,5,bl_cols,isflip)
        
        model_1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X1, Y1)
        model_3 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X3, Y3)
        model_5 = BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100).fit(X5, Y5)
        
        pred_map_1 = submit_predict(task_json,model_1,1)
        pred_map_3 = submit_predict(task_json,model_3,3)
        pred_map_5 = submit_predict(task_json,model_5,5)
        
        for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_3,pred_map_5):
            task_ids.append(tks)
            task_preds.append(str_pred)
            #print(tks,str_pred)
        solved_task+=1
        #break
    else:
        pred_map_1 = dumb_predict(task_json)
        pred_map_3 = dumb_predict(task_json)
        pred_map_5 = dumb_predict(task_json)
        
        for tks,str_pred in combine_preds(tk_id,pred_map_1,pred_map_3,pred_map_5):
            task_ids.append(tks)
            task_preds.append(str_pred)
            #print(tks,str_pred)
        
    total_task+=1
    
sample_sub1 = pd.DataFrame({"output_id":task_ids,'output':task_preds})
sample_sub1.to_csv("submission_1.csv", index=None)

In [ ]:
import os
import time
from pathlib import Path

KAGGLE = Path('/kaggle/working').is_dir()

In [ ]:
%%writefile program.cpp


#include <iostream>
#include <fstream>
#include <string>
#include <vector>
#include <queue>
#include <utility>
#include <stdio.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <algorithm>
#include <numeric>
#include <map>
#include <dirent.h>

using namespace std;


template <typename T>
vector<size_t> sort_indexes(const vector<T> &v) {

  vector<size_t> idx(v.size());
  iota(idx.begin(), idx.end(), 0);

  stable_sort(idx.begin(), idx.end(),
       [&v](size_t i1, size_t i2) {return v[i1] > v[i2];});

  return idx;
}

template <typename A, typename B>
void zip(
    const std::vector<A> &a, 
    const std::vector<B> &b, 
    std::vector<std::pair<A,B>> &zipped)
{
    for(size_t i=0; i<a.size(); ++i)
    {
        zipped.push_back(std::make_pair(a[i], b[i]));
    }
}

template <typename A, typename B>
void unzip(
    const std::vector<std::pair<A, B>> &zipped, 
    std::vector<A> &a, 
    std::vector<B> &b)
{
    for(size_t i=0; i<a.size(); i++)
    {
        a[i] = zipped[i].first;
        b[i] = zipped[i].second;
    }
}

bool IsPathExist(const std::string &s)
{
  struct stat buffer;
  return (stat (s.c_str(), &buffer) == 0);
}

class Grid {
    public:
        int nrows = 0;
        int ncols = 0;
        int **mat = NULL;
        
        Grid() {}
    
        Grid(const Grid& grid) {
            gridCopy(grid);
        }
    
        void gridCopy(const Grid& grid) {
            nrows = grid.nrows;
            ncols = grid.ncols;
            
            mat = new int*[nrows];
            for (int i = 0; i < nrows; ++i) {
                mat[i] = new int[ncols];
                for (int j = 0; j < ncols; ++j)
                    mat[i][j] = grid.mat[i][j];
            }
        }
        
        Grid(int nr, int nc) {
            gridBySize(nr,nc);
        }
    
        void gridBySize(int nr, int nc) {
            nrows = nr;
            ncols = nc;
            
            mat = new int*[nrows];
            for (int i = 0; i < nrows; ++i) {
                mat[i] = new int[ncols];
                for (int j = 0; j < ncols; ++j)
                    mat[i][j] = -1;
            }
        }
        
        Grid(std::vector<std::vector<int>> &rows) {
            nrows = rows.size();
            ncols = rows[0].size();
            
            mat = new int*[nrows];
            for (int i = 0; i < nrows; ++i) {
                mat[i] = new int[ncols];
                for (int j = 0; j < ncols; ++j)
                    mat[i][j] = rows[i][j];
            }
        }
        
        void print() {
            std::cout << "\n";
            for (int i = 0; i < nrows; ++i) {
                for (int j = 0; j < ncols; ++j) {
                    if (mat[i][j] == 0)
                        std::cout << '.';
                    else
                        std::cout << mat[i][j];
                }
                std::cout << "\n";
            }
        }
        
        ~Grid() {
            clear();
        }
    
        virtual void clear() {
            if (mat) {
                for (int i = 0; i < nrows; ++i)
                    delete mat[i];
                delete mat;
            }
            mat = NULL;
        }
        
        bool operator== (const Grid &ref) const
        {
            if ((nrows != ref.nrows) | (ncols != ref.ncols))
                return false;
            for (int i = 0; i < nrows; ++i)
                for (int j = 0; j < ncols; ++j)
                    if (mat[i][j] != ref.mat[i][j])
                        return false;
            return true;
        }
        
        std::string flatten() {
            std::string out;
            for (int i = 0; i < nrows; ++i) {
                out.append("|");
                for (int j = 0; j < ncols; ++j)
                    out.append(std::to_string(mat[i][j]));
            }
            out.append("|");
            return out;
        }
};

class Pair {
    public:
        Grid *input = NULL;
        Grid *output = NULL;
        
        Pair(Grid *inp, Grid *out) {
            input = inp;
            output = out;
        }
    
        Pair() {}
        
        void print() {
            input->print();
            output->print();
        }
    
        void set_input(Grid *inp) {
            input = inp;
        }
    
        void set_output(Grid *out) {
            output = out;
        }
        
        ~Pair() {
            if (input)
                delete input;
            if (output)
                delete output;
        }
};

class Task {
    public:
        std::vector<Pair*> train;
        std::vector<Pair*> test;
        
        void add_train(Pair* pair) {
            train.push_back(pair);
        }
    
        void add_test(Pair* pair) {
            test.push_back(pair);
        }
    
        ~Task() {
            for (Pair* pair : train)
                delete pair;
            for (Pair* pair : test)
                delete pair;
        }
};

class Property : public Grid {
    public:
        Property() {}
        Property(const Grid& grid) : Grid(grid) {}
        std::string name;
        virtual void populate(Grid *grid) {}
        virtual void pre(vector<Property*> props) {}
        virtual void post(vector<Property*> props) {}

        void calcBackground(Grid* grid) {

            int cnts[10] = {};
            for (int i = 0; i < grid->nrows; ++i)
                for (int j = 0; j < grid->ncols; ++j)
                    cnts[grid->mat[i][j]]++;

            int max_cnt = -1;
            for (int c = 0; c < 10; ++c)
                if (cnts[c] > max_cnt) {
                    max_cnt = cnts[c];
                    bg[0] = c;
                }
        }

        void copyBG(int* bg_in) {
            for (int i=0; i<10; i++)
                bg[i] = bg_in[i];
        }

        bool operator== (const Property &ref) const {
            return Grid::operator==((const Grid&)ref);
        }

        int bg[10] {};

        vector<pair<int,int>> dirs = {make_pair(-1,-1),make_pair(-1,0),make_pair(-1,1),make_pair(0,-1),
                                      make_pair(0,1),make_pair(1,-1),make_pair(1,0),make_pair(1,1)};
        vector<pair<int,int>> dirs_rook = {make_pair(-1,0),make_pair(0,1),make_pair(1,0),make_pair(0,-1)};
        vector<pair<int,int>> dirs_row = {make_pair(0,1),make_pair(0,-1)};
        vector<pair<int,int>> dirs_col = {make_pair(1,0),make_pair(-1,0)};
        vector<pair<int,int>> dirs_diag1 = {make_pair(1,1),make_pair(-1,-1)};
        vector<pair<int,int>> dirs_diag2 = {make_pair(1,-1),make_pair(-1,1)};
};

class PropBorderType : public Property {
    public:
    
        PropBorderType() {
            name = "PropBorderType";
        }
        
        virtual void populate(Grid *grid) {

            gridBySize(grid->nrows, grid->ncols);

            markCell(grid,0,0,1);
            markCell(grid,0,ncols-1,2);
            markCell(grid,nrows-1,0,3);
            markCell(grid,nrows-1,ncols-1,4);
            
            for (int i = 1; i < (nrows-1); ++i) {
                markCell(grid,i,0,5);
                markCell(grid,i,ncols-1,6);
            }
            for (int j = 1; j < (ncols-1); ++j) {
                markCell(grid,0,j,7);
                markCell(grid,nrows-1,j,8);
            }

            for (int i = 0; i < nrows; ++i)
                for (int j = 0; j < ncols; ++j)
                    if (mat[i][j] < 0) mat[i][j] = 0;
        }
    
    private:
        
        void markCell(Grid *grid, int i, int j, int col) {
            if (mat[i][j] >= 0) 
                return;
            mat[i][j] = col;
            int c = grid->mat[i][j];
            for (pair<int,int> p : dirs_rook) {
                int x = i + p.first;
                int y = j + p.second;
                if ((x < 0) || (x >= nrows) || (y < 0) || (y >= ncols))
                    continue;
                if (grid->mat[x][y] != c)
                    continue;
                markCell(grid, x, y, col);
            }
        }
};


class PropBorderDistance : public Property {
    public:
    
        PropBorderDistance(int max_dist = 100, bool onBG = true) : max_dist{max_dist}, onBG{onBG} {
            name = "PropBorderDistance";

            name += to_string(max_dist);
            if (onBG)
                name += "G";
        }
        
        virtual void populate(Grid *grid) {

            if (bg[0] == -1)
                calcBackground(grid);

            nrows = grid->nrows;
            ncols = grid->ncols;
            
            mat = new int*[nrows];
            for (int i = 0; i < nrows; ++i) {
                mat[i] = new int[ncols];
                for (int j = 0; j < ncols; ++j)
                    mat[i][j] = 0;
            }
            
            for (int i = 0; i < nrows; ++i) {
                markCell(grid,i,0,1);
                markCell(grid,i,ncols-1,1);
            }
            for (int j = 1; j < (ncols-1); ++j) {
                markCell(grid,0,j,1);
                markCell(grid,nrows-1,j,1);
            }
            
            while(search.size() > 0) {
                std::pair<int,int> p = search.front();
                search.pop();
                int depth = mat[p.first][p.second] + 1;
                if (depth > max_dist) depth = max_dist;
                markCell(grid,p.first-1,p.second,depth);
                markCell(grid,p.first+1,p.second,depth);
                markCell(grid,p.first,p.second-1,depth);
                markCell(grid,p.first,p.second+1,depth);
            }
        }
    
    private:
    
        std::queue<std::pair<int,int>> search;
        
        void markCell(Grid *grid, int i, int j, int depth) {
            if ((i < 0) | (i >= nrows) | (j < 0) | (j >= ncols))
                return;
            if ((!onBG || (grid->mat[i][j] == bg[0])) && (mat[i][j] == 0)) {
                mat[i][j] = depth;
                std::pair<int,int> p(i,j);
                search.push(p);
            }
        }

        int max_dist = 0;
        bool onBG = true;
};

class PropHoles : public PropBorderDistance {
    public:
        
        PropHoles() {
            name = "PropHoles";
        }
    
        virtual void populate(Grid *grid) {
            PropBorderDistance::populate(grid);
            for (int i = 0; i < nrows; ++i)
                for (int j = 0; j < ncols; ++j)
                    if ((mat[i][j] == 0) && (grid->mat[i][j] == 0))
                        mat[i][j] = 1;
                    else
                        mat[i][j] = 0;
        }
};

class PropColor : public Property {
    public:
    
        PropColor () {
            name = "PropColor";
        }
    
        virtual void populate(Grid *grid) {
            gridCopy(*grid);
        }
};
    
class PropModulo : public Property {
    public:
    
        PropModulo(int mod, bool is_row, bool is_col, bool is_rev) : mod{mod}, is_row{is_row}, is_col{is_col}, is_rev(is_rev) {
            name = "PropModulo";
            name += to_string(mod);
            if (is_row)
                name += "T";
            else
                name += "F";
            if (is_col)
                name += "T";
            else
                name += "F";
            if (is_rev)
                name += "R";
        }
    
        virtual void populate(Grid *grid) {
            gridCopy(*grid);
            
            for (int i = 0; i < nrows; ++i)
                for (int j = 0; j < ncols; ++j) {
                    mat[i][j] = 0;
                    int x = i;
                    int y = j;
                    if (is_rev) {
                        x = nrows-1-x;
                        y = ncols-1-y;
                    }
                    if (is_row)
                        mat[i][j] += mod*(x%mod);
                    if (is_col)
                        mat[i][j] += (y%mod);
                }
        }

    private:
        int mod = 2;
        bool is_row = true;
        bool is_col = true;
        bool is_rev = false;
};



class Action {
    public:
        std::string name;
        
        vector<vector<Grid*>> test_out;
        vector<string> names_out;

        virtual bool train(Task* task) {}
        virtual Grid* generate(Grid* grid, int test_id) {}
};


class Coloring {
    public:
        Coloring(std::vector<Property*> props, bool old_post=false) : props {props}, old_post{old_post} {
            len = props.size();
            constructor_init();
        }
        
        Coloring(bool old_post=false, int len=1) : old_post{old_post}, len{len} {
            constructor_init();
        }

        void constructor_init() {
            if (len < 2) m_separate = false;
            if (m_separate)
                for (int c1=0; c1<10; ++c1) {
                    for (int c2=0; c2<10; ++c2) {
                        sep_row[c1][c2] = -1;
                    }
                }
        }

        void init_props(std::vector<Property*> props_in) {
            props = props_in;
            init_props_low();
        }

        void init_props(Grid* grid) {
            for (Property* prop : props) {
                prop->clear();
                prop->populate(grid);
            }
            
            init_props_low();
        }

        void init_props_low() {
            for (int i=0; i<3; i++)
                if (i < len)
                    mx[i] = 10;
                else
                    mx[i] = 1;

            if (train_id == 0) {
                name = "";
                for (Property* prop : props)
                    name += prop->name + ";";
            }
        }
        
        int getVal(int i=0, int j=0, int z=0) {
            return (cmap[i][j][z] - 1);
        }
    
        int getVal(int* idx) {
            return getVal(idx[0],idx[1],idx[2]);
        }
    
        void setVal(int i=0, int j=0, int z=0, int val=0) {
            cmap[i][j][z] = val + 1;
        }
        
        void finalizeTrain(int id) {
            if (m_separate && (train_id == 0)) {
                for (int c3 = 0; c3 < mx[2]; ++c3) {
                    for (int c2 = 0; c2 < mx[1]; ++c2) {
                        int found_color = -1;
                        for (int c1 = 0; c1 < mx[0]; ++c1) {
                            if (getVal(c1,c2,c3) >= 0) {
                                if ((found_color >= 0)) {
                                    m_separate = false;
                                }
                                sep_row[c2][c3] = c1;
                                found_color = c1;
                            }
                        }
                    }
                }
            }
        }
        
        int inferenceSeparate(int i, int j) {
            int idx0 = props[0]->mat[i][j];
            int idx1 = props[1]->mat[i][j];
            int idx2 = 0;
            if (len == 3)
                idx2 = props[2]->mat[i][j];
            
            int cm = sep_map[idx0];
            if (cm == -1) return -1;
            cm = getVal(cm, idx1, idx2);
            if (cm == -1) return -1;
            cm = sep_map_rev[cm];
            
            return cm;
        }
    
        bool trainSeparate(int i, int j, int c_out) {
            
            if (train_id == 0) {
                bool ret = trainTogether(i,j,c_out);
                return ret;
            } else {
                int cm = inferenceSeparate(i,j);
                if (cm != c_out) return false;
                return true;
            }
        }
        
        bool trainTogether(int i, int j, int c_out) {
            int cm = get_color_base(i,j);
            
            if (cm >= 0) {
                if (cm != c_out)
                    return false;
            } else
                set_color(i,j,c_out);
            
            return true;
        }
        
        bool train(int i, int j, int c_out) {
            if (m_separate) {
                bool ret_separate = trainSeparate(i,j,c_out);
                m_separate = m_separate && ret_separate;
            }
            if (m_together) {
                bool ret_together = trainTogether(i,j,c_out);
                m_together = m_together && ret_together;
            }
            return (m_together || m_separate);
        }
        
        void setTrainId(int id) {
            train_id = id;
            if (m_separate) {
                for (int c=0; c<10; ++c) {
                    sep_map[c] = -1;
                    sep_map_rev[c] = -1;
                }

                if (train_id != 0) {
                    for (int i=0; i<props[0]->nrows; i++)
                        for (int j=0; j<props[0]->ncols; j++) {
                            int idx0 = props[0]->mat[i][j];
                            int idx1 = props[1]->mat[i][j];
                            int idx2 = 0;
                            if (len == 3)
                                idx2 = props[2]->mat[i][j];
                            sep_map[idx0] = sep_row[idx1][idx2];
                        }
                    
                    for (int c=0; c<10; ++c) {
                        if (sep_map[c] >= 0)
                            sep_map_rev[sep_map[c]] = c;
                    }
                }
            }

        }
        
        void printCMap() {
            if (len <= 2) {
                for (int c1 = 0; c1 < 10; ++c1) {
                    for (int c2 = 0; c2 < 10; ++c2) {
                        int c = getVal(c1,c2);
                        if (c >= 0)
                            std::cout << getVal(c1,c2);
                        else
                            std::cout << ".";
                    }
                    std::cout << "\n";
                }
            } else {
                for (int c1 = 0; c1 < 10; ++c1) {
                    std::cout << "first prop " << c1 << "\n";
                    for (int c2 = 0; c2 < 10; ++c2) {
                        for (int c3 = 0; c3 < 10; ++c3) {
                            int c = getVal(c1,c2,c3);
                            if (c >= 0)
                                std::cout << c;
                            else
                                std::cout << ".";
                        }
                        std::cout << "\n";
                    }
                    std::cout << "\n";
                }
            }

        }

        void post_process() {
            int cm;

            if (m_separate) {
                name += "SEP;";
            }
            
            int cols[3][10][2][10] = {};
            int diag[3][10] = {};

            for (int i = 0; i < 3; ++i) {
                for (int c = 0; c < mx[i]; ++c) {
                    for (int c1 = 0; c1 < mx[dims[i][0]]; ++c1) {
                        for (int c2 = 0; c2 < mx[dims[i][1]]; ++c2) {
                            int idx[3];
                            idx[i] = c;
                            idx[dims[i][0]] = c1;
                            idx[dims[i][1]] = c2;
                            cm = getVal(idx);
                            if (cm >= 0) {
                                cols[i][c][0][c1]++;
                                cols[i][c][1][c2]++;
                                if (c1 == c2)
                                    diag[i][c]++;
                            }
                        }
                    }
                }
            }
            
            while (true) {

                int max_cnt = 0;
                for (int i = 0; i < 3; ++i) {
                    for (int c = 0; c < mx[i]; ++c) {
                        for (int j = 0; j < 2; ++j) {
                            for (int c1 = 0; c1 < 10; ++c1) {
                                int val = cols[i][c][j][c1];
                                if ((val > max_cnt) && (val < 10)) max_cnt = val;
                            }
                        }
                        int val = diag[i][c];
                        if ((val > max_cnt) && (val < 10)) max_cnt = val;
                    }
                }

                int vals[10][3];
                bool found = false;

                for (int i = 0; !found && (i < 3); ++i) {
                    for (int c = 0; !found && (c < mx[i]); ++c) {
                        for (int j = 0; !found && (j < 2); ++j) {
                            for (int c1 = 0; !found && (c1 < 10); ++c1) {
                                int val = cols[i][c][j][c1];
                                if (val == max_cnt) {
                                    int idx[3];
                                    idx[0] = i;
                                    idx[1] = dims[i][j];
                                    idx[2] = dims[i][1-j];
                                    for (int c2 = 0; c2 < 10; ++c2) {
                                        vals[c2][idx[0]] = c;
                                        vals[c2][idx[1]] = c1;
                                        vals[c2][idx[2]] = c2;
                                    }
                                    found = true;
                                }
                            }
                        }
                        int val = diag[i][c];
                        if (val == max_cnt) {
                            int idx[3];
                            idx[0] = i;
                            idx[1] = dims[i][0];
                            idx[2] = dims[i][1];
                            for (int c2 = 0; c2 < 10; ++c2) {
                                vals[c2][idx[0]] = c;
                                vals[c2][idx[1]] = c2;
                                vals[c2][idx[2]] = c2;
                            }
                            found = true;
                        }
                    }
                }

                if (!found) break;

                bool any_colored = false;

                int selfColors[2] = {};
                float cnts[10] = {};
                for (int i = 0; i < 10; ++i) {
                    cm = getVal(vals[i][0],vals[i][1],vals[i][2]);
                    if (cm >= 0) {
                        selfColors[int(cm == i)]++;
                        if (vals[i][1] == vals[i][2])
                            cnts[cm] += 1.0;
                        else
                            cnts[cm] += 1.05;
                    }
                }
                int numColors = 0;
                int freqColor = 0;
                for (int i = 0; i < 10; ++i) {
                    if (cnts[i] > cnts[freqColor])
                        freqColor = i;
                    numColors += int(cnts[i] > 0);
                }
                bool selfColoring = ((selfColors[0] == 0) && (selfColors[1] >= 2)) ||
                                    ((selfColors[0] == 1) && (selfColors[1] >= 3));
                for (int i = 0; i < 10; ++i) {
                    if (getVal(vals[i][0],vals[i][1],vals[i][2]) < 0) {
                        int col = -1;
                        if (selfColoring)
                            col = i;
                        else if ((numColors <= 2) && (cnts[freqColor] >= 2.0))
                            col = freqColor;
                        
                        if (col >= 0) {
                            setVal(vals[i][0],vals[i][1],vals[i][2],col);
                            any_colored = true;

                            for (int j = 0; j < 3; ++j) {
                                cols[j][vals[i][j]][0][vals[i][dims[j][0]]]++;
                                cols[j][vals[i][j]][1][vals[i][dims[j][1]]]++;
                                if (vals[i][dims[j][0]] == vals[i][dims[j][1]])
                                    diag[j][vals[i][j]]++;
                            }
                        }
                    }
                }

                if (!any_colored) break;
            }
        }
        
        void set_color(int i, int j, int val) {
            int idx0 = props[0]->mat[i][j];
            int idx1 = 0;
            int idx2 = 0;
            if (len >= 2)
                idx1 = props[1]->mat[i][j];
            if (len >= 3)
                idx2 = props[2]->mat[i][j];
            setVal(idx0,idx1,idx2,val);
        }

        int get_color(int i, int j) {
            if (m_separate)
                return inferenceSeparate(i,j);
            else
                return get_color_base(i,j);
        }

        int get_color_base(int i, int j) {
            int idx0 = props[0]->mat[i][j];
            int idx1 = 0;
            int idx2 = 0;
            if (len >= 2)
                idx1 = props[1]->mat[i][j];
            if (len >= 3)
                idx2 = props[2]->mat[i][j];
            return getVal(idx0,idx1,idx2);
        }

        string name;
        bool m_separate = true;

    private:
        int cmap[10][10][10] = {};
        int mx[3] = {};
        int dims[3][2] = {{1,2}, {0,2}, {0,1}};
        int len = 0;
        int train_id = 0;
        bool m_together = true;
        int sep_map[10];
        int sep_map_rev[10];
        int sep_row[10][10];
        bool old_post;
        std::vector<Property*> props;    
};


Task* parse_file(std::string filepath) {
    
    std::ifstream ifs;
    ifs.open(filepath.c_str(), std::ifstream::in);
    
    int cnt = 0;
    bool char_t = false;
    bool char_n = false;
    bool is_train = false;
    bool is_test = false;
    bool is_input = false;
    bool is_output = false;
    int opened = 0;
    int opened_curl = 0;
    
    std::vector<int> row;
    std::vector<std::vector<int>> rows;
    
    std::vector<Pair> pairs;
    Pair *pair = NULL;
    Task *task = new Task();
    
    char c = ifs.get();
    while (ifs.good()) {
        
        if ((c == 'e') & char_t)
            is_test = true;
        if ((c == 'r') & char_t)
            is_train = true;
        
        if ((c == 'p') & char_n)
            is_input = true;
        if ((c == 'p') & char_t)
            is_output = true;
        
        char_t = (c == 't');
        char_n = (c == 'n');
        
        if (c == '{') {
            opened_curl++;
            if (opened_curl == 2)
                pair = new Pair();
        }
        if (c == '}') {
            opened_curl--;
        }
        
        if (c == '[') {
            opened++;
        }
        if (c == ']') {
            opened--;
            if (opened == 0) {
                is_test = false;
                is_train = false;
            }
            if (opened == 1) {
                Grid *grid = new Grid(rows);
                
                if (is_input) {
                    pair->set_input(grid);
                
                    if (is_train)
                        task->add_train(pair);
                    if (is_test)
                        task->add_test(pair);
                }
                if (is_output) {
                    pair->set_output(grid);
                }
                
                rows.clear();
                
                is_input = false;
                is_output = false;
            }
            if (opened == 2) {
                rows.push_back(row);
                row.clear();
            }
        }
        
        if ((c >= '0') & (c <= '9')) {
            row.push_back(int(c) - int('0'));
        }
        
        c = ifs.get();
        cnt++;
    }

    ifs.close();
    
    return task;
}

class PropSet {
    public:
        PropSet(vector<Property*> &props) : props{props} {}

        Property* operator[](int index) { 
            return props[index]; 
        }

        void post() {

            int cnts[10] {};
            for (Property* p : props) {
                for (int i=0; i<p->nrows; ++i)
                    for (int j=0; j<p->ncols; ++j)
                        cnts[p->mat[i][j]]++;
            }

            num_classes = 0;
            for (int c=0; c<10; ++c)
                if (cnts[c] > 0)
                    num_classes++;
        }

        vector<Property*> props;
        int num_classes = 0;
};

class PropManager {
    public:

        PropManager(Task *task) : task{task} {
            bg = calcBackground(task);
        }

        void calcBackgroundCount(Grid* grid, int* cnts) {
            for (int i = 0; i < grid->nrows; ++i)
                for (int j = 0; j < grid->ncols; ++j)
                    cnts[grid->mat[i][j]]++;
        }

        int* calcBackground(Task* task) {
            int cnts[10] = {};

            for (Pair* pair : task->train)
                calcBackgroundCount(pair->input, cnts);
            
            for (Pair* pair : task->test)
                calcBackgroundCount(pair->input, cnts);

            int* bg = new int[10] {};
            
            std::vector<int> cnts_vec;
            cnts_vec.assign(cnts, cnts + 10);
            int k=0;
            for (auto i: sort_indexes(cnts_vec)) {
                bg[k] = i;
                k++;
            }

            return bg;
        }

        template<class PType, typename ... Args> void addProperty(Args ... args) {

            vector<Property*> task_props;

            int train_id = 0;
            for (Pair* pair : task->train) {
                Property* prop = (Property*) new PType(args...);
                prop->copyBG(bg);
                prop->populate(pair->input);
                task_props.push_back(prop);
                train_id++;
            }

            vector<Property*> task_props_test;

            for (Pair* pair : task->test) {
                Property* prop = (Property*) new PType(args...);
                prop->copyBG(bg);
                prop->populate(pair->input);
                task_props_test.push_back(prop);
            }

            vector<Property*> task_props_all;
            for (Property* p : task_props)
                task_props_all.push_back(p);
            for (Property* p : task_props_test)
                task_props_all.push_back(p);

            task_props_all[0]->post(task_props_all);

            if (false) {
                for (Property* prop : task_props) {
                    cout << "DEBUG: " << prop->name << "\n";
                    prop->print();
                }
                for (Property* prop : task_props_test) {
                    cout << "DEBUG TEST: " << prop->name << "\n";
                    prop->print();
                }
            }

            PropSet prop_set(task_props);
            prop_set.post();
            if (prop_set.num_classes == 1) {
                return;
            }

            bool different = true;
            int k=0;
            for (PropSet saved_props : props) {
                different = false;
                int i = 0;
                for (Property* saved_prop : saved_props.props) {
                    if (!((*saved_prop) == (*task_props[i]))) {
                        different = true;
                        break;
                    }
                    i++;
                }
                i = 0;
                for (Property* saved_prop : props_test[k].props) {
                    if (!((*saved_prop) == (*task_props_test[i]))) {
                        different = true;
                        break;
                    }
                    i++;
                }
                k++;
                if (!different) {
                    break;
                }
            }

            if (!different) return;

            props.push_back(prop_set);
            names.push_back(task_props[0]->name);
            props_test.push_back(PropSet(task_props_test));
        }

        void print() {
            for (string name : names)
                cout << name << "\n";
        }

        vector<PropSet> props;
        vector<PropSet> props_test;
        vector<string> names;
        Task* task;
        int* bg = NULL;
};


class MultiColoring {
    public:
        MultiColoring(PropManager *mngr) : mngr{mngr} {
        }
        
        ~MultiColoring() {
            for (vector<Coloring*> vcol : colorings)
                for (Coloring* col : vcol)
                    delete col;
            
            for (int i=0; i<3; i++)
                delete failed[i];
        }

        void init(int level_in) {
            
            level = level_in;
            vector<Coloring*> colorings_lvl;
            vector<int> scores_lvl;

            if (level == 0) {
                for (PropSet vp : mngr->props) {
                    Coloring* coloring = new Coloring(false, 1);
                    colorings_lvl.push_back(coloring);
                    scores_lvl.push_back(vp.num_classes);
                }
            } else if (level == 1) {
                int i=0;
                for (PropSet vp1 : mngr->props) {
                    int k=0;
                    for (PropSet vp2 : mngr->props) {
                        if (k > i) {
                            Coloring* coloring = new Coloring(false, 2);
                            colorings_lvl.push_back(coloring);
                            scores_lvl.push_back(vp1.num_classes + vp2.num_classes);
                        }
                        k++;
                    }
                    i++;
                }
            } else {
                int p=0;
                for (PropSet vp0 : mngr->props) {
                    int i=0;
                    for (PropSet vp1 : mngr->props) {
                        int k=0;
                        for (PropSet vp2 : mngr->props) {
                            if ((k > i) && (i > p)) {
                                Coloring* coloring = new Coloring(false, 3);
                                colorings_lvl.push_back(coloring);
                                scores_lvl.push_back(vp0.num_classes + vp1.num_classes + vp2.num_classes);
                            }
                            k++;
                        }
                        i++;
                    }
                    p++;
                }
            }

            colorings.push_back(colorings_lvl);
            scores.push_back(scores_lvl);

            failed[level] = new bool[colorings[level].size()] {};
        }

        void setId(vector<PropSet> props_in, int id) {
            if (level == 0) {
                int i = 0;
                for (PropSet vp : props_in) {
                    std::vector<Property*> props;
                    props.push_back(vp[id]);
                    colorings[level][i]->init_props(props);
                    i++;
                }
            } else if (level == 1) {
                int num = 0;
                int i = 0;
                for (PropSet vp1 : props_in) {
                    int k=0;
                    for (PropSet vp2 : props_in) {
                        if (k > i) {
                            std::vector<Property*> props;
                            props.push_back(vp1[id]);
                            props.push_back(vp2[id]);
                            colorings[level][num]->init_props(props);
                            num++;
                        }
                        k++;
                    }
                    i++;
                }
            } else {
                int num = 0;
                int p=0;
                for (PropSet vp0 : props_in) {
                    int i = 0;
                    for (PropSet vp1 : props_in) {
                        int k=0;
                        for (PropSet vp2 : props_in) {
                            if ((k > i) && (i > p)) {
                                std::vector<Property*> props;
                                props.push_back(vp0[id]);
                                props.push_back(vp1[id]);
                                props.push_back(vp2[id]);
                                colorings[level][num]->init_props(props);
                                num++;
                            }
                            k++;
                        }
                        i++;
                    }
                    p++;
                }

            }
        }

        void setTrainId(int train_id) {
            setId(mngr->props, train_id);
            for (Coloring* col : colorings[level])
                col->setTrainId(train_id);
        }

        void setTestId(int test_id) {
            setId(mngr->props_test, test_id);
            for (Coloring* col : colorings[level])
                col->setTrainId(-1);
        }

        bool train(int i, int j, int c) {
            int k = -1;
            bool any_success = false;
            for (Coloring* colors : colorings[level]) {
                k++;
                if (failed[level][k])
                    continue;
                bool ret = colors->train(i, j, c);
                if (!ret) {
                    failed[level][k] = true;
                } else any_success = true;
            }
            return any_success;
        }

        void finalizeTrain(int id) {
            for (Coloring* colors : colorings[level]) {
                colors->finalizeTrain(id);
            }
        }

        bool post_process(Task* task) {
            vector<int> selected_scores;
            for (int i=0; i<colorings[level].size(); i++)
                if (!failed[level][i]) {
                    Coloring* selected_col = colorings[level][i];
                    selected_col->post_process();
                    selected_cols.push_back(selected_col);
                    selected_scores.push_back(scores[level][i]);
                }
            
            if (selected_cols.size() > 0) {
                std::vector<std::pair<Coloring*,int>> zipped;
                zip(selected_cols, selected_scores, zipped);

                std::sort(std::begin(zipped), std::end(zipped), 
                    [&](const auto& a, const auto& b)
                    {
                        return a.second < b.second;
                    });

                unzip(zipped, selected_cols, selected_scores);
            }

            return (selected_cols.size() > 0);
        }

        vector<Coloring*> selected_cols;

    private:
        
        vector<vector<Coloring*>> colorings;
        vector<vector<int>> scores;
        bool* failed[3];
        int level = 0;
        
        PropManager *mngr = NULL;
};


class ActMultiProps : public Action {
    public:
        ActMultiProps(PropManager *mngr) : mngr {mngr} {
            name = "ActMultiProps";
            colors = new MultiColoring(mngr);
        }
        
        ~ActMultiProps() {
            delete colors;
        }
        
        bool train(Task* task) {

            for (int level=0; level<3; level++) {
                
                colors->init(level);
                int train_id = 0;

                bool failed = false;
                for (Pair* pair : task->train) {
                    if (level == 0)
                        if ((pair->input->nrows != pair->output->nrows) || (pair->input->ncols != pair->output->ncols))
                            return false;
                    
                    colors->setTrainId(train_id);
                    
                    for (int i = 0; i < pair->input->nrows; ++i) {
                        for (int j = 0; j < pair->input->ncols; ++j) {
                            int c = pair->output->mat[i][j];
                            bool ret = colors->train(i, j, c);
                            if (ret == false) {
                                failed = true;
                                break;
                            }
                        }
                        if (failed) break;
                    }
                    if (failed) break;
                    
                    colors->finalizeTrain(train_id);
                    
                    train_id++;
                }
                
                if (!failed) {
                    bool success = colors->post_process(task);
                    
                    if (!success)
                        continue;

                    success = false;
                    for (Coloring *col: colors->selected_cols) {
                        int test_id = 0;
                        vector<Grid*> out_vec;
                        for (Pair* pair : task->test) {
                            Grid* out = generateTest(pair->input, test_id, col);
                            if (out == NULL) {
                                for (Grid* gg : out_vec)
                                    delete gg;
                                out_vec.clear();
                                break;
                            }
                            out_vec.push_back(out);
                            test_id++;
                        }
                        if (out_vec.size() > 0) {
                            success = true;
                            test_out.push_back(out_vec);
                            names_out.push_back(col->name);
                        }
                    }

                    if (success) return true;
                    else colors->selected_cols.clear();
                }
            }
            
            return false;
        }

        Grid* generateTest(Grid* grid, int test_id, Coloring* col) {
            Grid* out = new Grid(*grid);
            
            colors->setTestId(test_id);
            
            for (int i = 0; i < grid->nrows; ++i)
                for (int j = 0; j < grid->ncols; ++j) {
                    int c = col->get_color(i,j);
                    if (c < 0) {
                        delete out;
                        return NULL;
                    }
                    out->mat[i][j] = c;
                }
            
            return out;
        }
        
        Grid* generate(Grid* grid, int test_id) {
            return NULL;
        }

    private:
        PropManager *mngr = NULL;
        MultiColoring *colors = NULL;
};


int main(int argc, char *argv[]) {
    
    bool KAGGLE = IsPathExist("/kaggle/working/");
    
    std::string current_exec_name = argv[0];
    std::vector<std::string> all_args;
    
    std::string folder = std::string("test");
    if (argc > 1) {
        all_args.assign(argv + 1, argv + argc);
        folder = all_args[0];
    } else {
        if (!KAGGLE) {
            folder = std::string("training");
            //folder = std::string("evaluation");
        }
    }
    
    std::string PATH = "C:\\StudioProjects\\ARC\\";
    std::string PATH_WORK = "C:\\StudioProjects\\ARC\\";
    std::string SPLITTER = "\\";
    if (KAGGLE) {
        PATH = "/kaggle/input/abstraction-and-reasoning-challenge/";
        PATH_WORK = "/kaggle/working/";
        SPLITTER = "/";
    }
    
    std::cout << "Start" << "\n";
    if (KAGGLE)
        std::cout << "Running Kaggle" << "\n";
    else
        std::cout << "Running Local, folder " << folder << "\n";
    
    std::string path = PATH + folder;
    std::string sub_path = PATH_WORK + std::string("submission.csv");
    std::string succ_path = PATH_WORK + std::string("success.csv");
    
    std::ofstream ifs_sub;
    ifs_sub.open(sub_path.c_str(), std::ofstream::out);
    ifs_sub << "output_id,output\n";

    std::ifstream ifs_succ;
    ifs_succ.open(succ_path.c_str(), std::ifstream::in);
    vector<string> succ_ids;
    std::string line;
    while (getline(ifs_succ, line)) {
        succ_ids.push_back(line);
    }
    ifs_succ.close();

    std::ofstream ofs_succ;
    ofs_succ.open(succ_path.c_str(), ios_base::app | std::ofstream::out);

    std::map<std::string,std::string> description;
    description[std::string("3bdb4ada")]=std::string("LATER: grid in grid");
    description[std::string("63613498")]=std::string("HARD: color sharing between shapes");
    description[std::string("a5f85a15")]=std::string("TODO: modulo2 column");
    description[std::string("bda2d7a6")]=std::string("LATER: separate coloring harder case");
    description[std::string("0692e18c")]=std::string("TODO: zoom-in with reversed colors");
    description[std::string("1da012fc")]=std::string("HARD: color sharing between shapes");
    description[std::string("45737921")]=std::string("TODO: property of adjacent color");
    description[std::string("62ab2642")]=std::string("TODO: smallest and biggest shape property");
    description[std::string("1caeab9d")]=std::string("LATER: shapes moving");
    description[std::string("ba97ae07")]=std::string("LATER: need 3 props?");
    description[std::string("d037b0a7")]=std::string("TODO: neighbor up one prop");
    description[std::string("f823c43c")]=std::string("TODO: denoising");
    description[std::string("aedd82e4")]=std::string("TODO: single point shape");
    description[std::string("0a2355a6")]=std::string("TODO: count holes inside");
    description[std::string("694f12f3")]=std::string("TODO: shape size, ordered per grid");
    description[std::string("b230c067")]=std::string("TODO: frequency of appearance of a shape");
    description[std::string("9565186b")]=std::string("LATER: lvl1 is not enough, need to go into lvl2");
    description[std::string("150deff5")]=std::string("HARD: fit two shapes into a complex shape");
    description[std::string("1e0a9b12")]=std::string("LATER: cells moving down");
    description[std::string("22eb0ac0")]=std::string("TODO: color to the left/right");
    description[std::string("25ff71a9")]=std::string("LATER: shapes moving one down");


    DIR *dir;
    struct dirent *ent;
    int n_task_ids = 0;

    if ((dir = opendir(path.c_str())) != NULL) {
        int score[3] = {}; // correct, error, skipped
        
        bool first = true;
        
        int* res_train = NULL;
        int* res_test = NULL;
        int sz = -1;
        
        while ((ent = readdir(dir)) != NULL) {
            std::string fn = std::string(ent->d_name);
            if ((fn == std::string(".")) | (fn == std::string("..")))
                continue;
            //if (fn != std::string("9def23fe.json")) continue;
            std::string filepath = PATH + folder + SPLITTER + fn;
            //std::cout << filepath << "\n";
            Task *task = parse_file(filepath);

            PropManager pm(task);
            pm.addProperty<PropColor>();
            pm.addProperty<PropHoles>();
            pm.addProperty<PropModulo>(2, true, true, false);
            pm.addProperty<PropBorderType>();            
            
            std::vector<Action*> acts {
                new ActMultiProps(&pm),
            };

            if (res_train == NULL) res_train = new int[acts.size()] {};
            if (res_test == NULL) res_test = new int[acts.size()] {};
            if (sz == -1) sz = acts.size();

            bool* success_train = new bool[acts.size()];
            bool any_success_train = false;
            
            if (first) {
                std::cout << "\n";
                for (int a=0; a<acts.size(); ++a)
                    std::cout << a << ". " << acts[a]->name << "\n";
                std::cout << "\n";
            }
            
            for (int a=0; a<acts.size(); ++a) {
                success_train[a] = acts[a]->train(task);
                any_success_train = any_success_train || success_train[a];
            }
            
            int test_id = 0;
            for (Pair* pair : task->test) {
                
                vector<Grid*> outputs;

                bool* success_test = new bool[acts.size()] {};
                bool any_success_test = false;

                n_task_ids++;
                std::string id_name = fn.substr(0,8) + "_" + std::to_string(test_id);

                Grid* out = NULL;
                int printed = 0;
                if (pair->output == NULL)
                    ifs_sub << id_name << ",";

                for (int a=0; a<acts.size(); ++a) {

                    int p=0;
                    for (vector<Grid*> out_vec : acts[a]->test_out) {
                        if (printed < 3) {

                            out = out_vec[test_id];
                            bool identical = false;
                            for (Grid* o : outputs)
                                if (*o == *out) {
                                    identical = true;
                                }
                            if (identical) {
                                p++;
                                continue;
                            }

                            outputs.push_back(out);

                            cout << acts[a]->names_out[p] << "\n";
                            if (pair->output != NULL) {
                                bool result = ((*out) == (*pair->output));
                                if (result && !success_test[a]) {
                                    success_test[a] = true;
                                    any_success_test = true;
                                }
                            } else {
                                if (printed > 0) {
                                    ifs_sub << " ";
                                    cout << id_name << "\n";
                                }
                                ifs_sub << out->flatten();
                            }
                            printed++;
                            p++;
                        }
                    }
                }

                any_success_train = false;
                for (int a=0; a<acts.size(); ++a)
                    any_success_train = any_success_train || success_train[a];
                
                if (pair->output == NULL) {
                    if (printed == 0)
                        ifs_sub << pair->input->flatten();
                    ifs_sub << "\n";
                }

                if (any_success_test) {
                    if (std::find(succ_ids.begin(), succ_ids.end(), id_name) == succ_ids.end())
                        ofs_succ << id_name << "\n";
                } else {
                    if (std::find(succ_ids.begin(), succ_ids.end(), id_name) != succ_ids.end())
                        cout << "achtung: " << id_name << "\n";
                }
                
                if (any_success_train) {
                    std::cout << id_name << " ";
                    for (int a=0; a<acts.size(); ++a)
                        std::cout << success_train[a] + success_test[a];
                    if (any_success_test)
                        std::cout << " > PASS    ";
                    else {
                        std::cout << " > FAILED  ";
                    
                        map<string,string>::iterator it = description.find(fn.substr(0,8));
                        if (it != description.end())
                            cout << ":" << it->second;
                    }

                    std::cout << "\n";
                }
                
                if (any_success_test && any_success_train)
                    score[0]++;
                if (!any_success_test && any_success_train)
                    score[1]++;
                if (!any_success_test && !any_success_train)
                    score[2]++;
                
                for (int a=0; a<acts.size(); ++a) {
                    res_train[a] += success_train[a];
                    res_test[a] += success_test[a];
                }

                test_id++;
            }
            
            for (int a=0; a<acts.size(); ++a) {
                delete acts[a];
            }
            delete success_train;
            delete task;
            
            first = false;
        }
        
        char buffer [50];
        cout << "\n";
        cout << "         ";
        for (int a=0; a<sz; ++a) {
            sprintf(buffer,"%3d",a);
            cout << buffer;
        }
        cout << "\n";
        cout << "train    ";
        for (int a=0; a<sz; ++a) {
            sprintf(buffer,"%3d",res_train[a]);
            cout << buffer;
        }
        cout << "\n";
        cout << "test     ";
        for (int a=0; a<sz; ++a) {
            sprintf(buffer,"%3d",res_test[a]);
            cout << buffer;
        }
        cout << "\n";

        delete res_train;
        delete res_test;

        std::cout << "correct: " << score[0] << " error: " << score[1] << " skipped: " << score[2] << "\n";
        
        closedir(dir);
    } else {
        perror("failed to read files");
        return 1;
    }
    
    ifs_sub.close();
    ofs_succ.close();

    cout << "n_task_ids: " << n_task_ids << "\n";
    if ((n_task_ids != 102) && (n_task_ids != 104)) {
        remove(sub_path.c_str());
    }

    return 0;
}

In [ ]:
st = time.time()
if KAGGLE:
    print(os.system("g++ -pthread -lpthread -O3 -std=c++17 -o main program.cpp 2> error.log"))
else:
    print(os.system("g++ -pthread -lpthread -g -std=c++17 -o main program.cpp 2> error.log"))
print("running time:", time.time()-st)

In [ ]:
st = time.time()
if KAGGLE:
    !./main training
else:
    !main.exe training
print("running time:", time.time()-st)

In [ ]:
st = time.time()
if KAGGLE:
    !./main evaluation
else:
    !main.exe evaluation
print("running time:", time.time()-st)

In [ ]:
st = time.time()
if KAGGLE:
    !./main test
else:
    !main.exe test
print("running time:", time.time()-st)

In [ ]:
sample_sub1 = sample_sub1.reset_index()
sample_sub1 = sample_sub1.sort_values(by="output_id")

submission = pd.read_csv("../input/dataset/submission-.980_c.csv")
sample_sub2 = submission.sort_values(by="output_id")
out1 = sample_sub1["output"].astype(str).values
out2 = sample_sub2["output"].astype(str).values

merge_output = []
for o1, o2 in zip(out1, out2):
    o = o1.strip().split(" ")[:1] + o2.strip().split(" ")[:2]
    o = " ".join(o[:3])
    merge_output.append(o)
sample_sub1["output"] = merge_output
sample_sub1["output"] = sample_sub1["output"].astype(str)
sample_sub1 = sample_sub1.drop(['index'], axis=1)
sample_sub1.to_csv("submission.csv", index=False)